In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [ ]:
#载入数据集
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

#定义批次大小 每次训练放入图片数量
batch_size=100

#计算一共多少个批次 (图片数量除以（//)批次大小）
n_batch=mnist.train.num_examples // batch_size

#运行次数
max_steps=201

#图片数量
image_num=3000

#文件路径 （程序所在路径）
DIR="/Users/yunhuarong/"

#定义会话
sess=tf.Session()

#载入图片
embedding=tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaires'):
        mean=tf.reduce_mean(var)
        #平均值
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        
#命名空间
with tf.name_scope('input'):
    #None 表示第一个维度为任意长度
    x=tf.placeholder(tf.float32,[None,784],name='x-input')
    #正确的标签
    y=tf.placeholder(tf.float32,[None,10],name='y-input')    
    keep_prob=tf.placeholder(tf.float32,name='kp')
    lr=tf.Variable(0.01,dtype=tf.float32,name='learning_rate')
    
#显示图片
with tf.name_scope('input_reshape'):
        image_shaped_input=tf.reshape(x,[-1,28,28,1])
        tf.summary.image('input',image_shaped_input,10)
        
#创建简单神经网络第一层
with tf.name_scope('layer1'):

    with tf.name_scope('weights1'):
        W1=tf.Variable(tf.truncated_normal([784,500],stddev=0.1),name='W1')
        variable_summaries(W1)
    with tf.name_scope('biases1'):    
        b1=tf.Variable(tf.zeros([500])+0.1,name='b1')
        variable_summaries(b1)
    with tf.name_scope('w1x_plus_b1'):    
        w1x_plus_b1=tf.matmul(x,W1)+b1
    with tf.name_scope('tanh1'):
        L1=tf.nn.tanh(w1x_plus_b1)  
        L1_drop=tf.nn.dropout(L1,keep_prob)     
        
#创建神经第二层
with tf.name_scope('layer2'):

    with tf.name_scope('weights2'):
        W2=tf.Variable(tf.truncated_normal([500,300],stddev=0.1),name='W2')
        variable_summaries(W2)
    with tf.name_scope('biases2'):    
        b2=tf.Variable(tf.zeros([300])+0.1,name='b2')
        variable_summaries(b2)
    with tf.name_scope('w2x_plus_b2'):    
        w2x_plus_b2=tf.matmul(L1,W2)+b2
    with tf.name_scope('tanh2'):
        L2=tf.nn.tanh(w2x_plus_b2)  
        L2_drop=tf.nn.dropout(L2,keep_prob)  
        
#定义输出层
with tf.name_scope('layer_out'):

    with tf.name_scope('weights_f'):
        Wf=tf.Variable(tf.truncated_normal([300,10],stddev=0.1),name='Wf')
        variable_summaries(Wf)
    with tf.name_scope('biases_f'):    
        bf=tf.Variable(tf.zeros([10])+0.1,name='bf')
        variable_summaries(bf)
    with tf.name_scope('prediction'):    
        prediction=tf.nn.softmax(tf.matmul(L2_drop,Wf)+bf)

with tf.name_scope('loss'):
#定义交叉墒代价函数
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)

with tf.name_scope('train'):
#训练
    train_step=tf.train.AdamOptimizer(lr).minimize(loss)

#初始化变量

init=tf.global_variables_initializer()

with tf.name_scope('accuracy'):
          with tf.name_scope('correct_prediction'):
#结果存放在布尔型列表中  定义准确率 比较两个agrmax大小，相同就是TRUE 不同就是FALSE。 argmax求最大值所在位置
                correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))

          with tf.name_scope('accuracy'):             
#求准确率 (tf.cast()将布尔型数据转化成数值型 然后计算准确率
                accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
                tf.summary.scalar('accuracy',accuracy)

#产生metadata文件
#检查指定路径是否有此文件，如果有删除
if tf.gfile.Exists(DIR+'projector/projector/metadata.tsv'):
    tf.gfile.DeleteRecursively(DIR+'projector/projector/metadata.tsv')
with open(DIR+'/projector/projector/metadata.tsv','w')as f:
    #查找1所在位置，提取位置就是标签
    labels=sess.run(tf.argmax(mnist.test.labels[:],1))
        #循环
    for i in range(image_num):
            #每次写入后隔开一行
            f.write(str(labels[i])+'\n')
            
#合并所有summaries
merged=tf.summary.merge_all()

#官网visualization例子
projector_writer=tf.summary.FileWriter(DIR+'projector/projector',sess.graph)
saver=tf.train.Saver()
config=projector.ProjectorConfig()
embed=config.embeddings.add()
embed.tensor_name=embedding.name
embed.metadata_path=DIR +'projector/projector/metadata.tsv'
embed.sprite.image_path=DIR +'projector/data/mnist_10k_sprite.png'
embed.sprite.single_image_dim.extend([28,28])
projector.visualize_embeddings(projector_writer,config)

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(max_steps):
        sess.run(tf.assign(lr,0.01*(0.95**i)))
        for batch in range (n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            run_options=tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            run_metadata=tf.RunMetadata() 
            summary,_=sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys,keep_prob:1.0},options=run_options,run_metadata=run_metadata)
            learning_rate=sess.run(lr)
        projector_writer.add_run_metadata(run_metadata,'step%03d'% i)
        projector_writer.add_summary(summary,i)
    
        if i%100 == 0:
            acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
            print("Iter"+str(i)+", Testing accuracy="+str(acc))
            
    saver.save(sess,DIR +'projector/projector/a_model.ckpt',global_step=max_steps)
    projector_writer.close()
sess.close()